<a href="https://colab.research.google.com/github/ecloguehwang/HSS/blob/master/%EB%84%A4%EC%9D%B4%EB%B2%84%EB%B6%80%EB%8F%99%EC%82%B0%EC%97%B0%EB%9D%BD%EC%B2%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

In [ ]:
#Deepseek로 수정한 코드

import pandas as pd
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import time

def get_data(article_no):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)

    base_url = 'https://new.land.naver.com/complexes/11337?ms=37.556255,126.840704,17&a=APT:ABYG:JGC:PRE&e=RETAIL&ad=true&articleNo='
    url = f"{base_url}{article_no}"

    driver.get(url)
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    phone_numbers = [dd.get_text() for dd in soup.find_all('dd', class_='text text--number')]
    real_estate_offices = [strong.get_text() for strong in soup.find_all('strong', class_='info_title')]

    apartment_info = []
    for h4 in soup.find_all('h4', class_='info_title'):
        title_name = h4.find('strong', class_='info_title_name').get_text() if h4.find('strong', class_='info_title_name') else ''
        floor_info = h4.get_text(strip=True).replace(title_name, '')
        apartment_info.append(f"{title_name} {floor_info}")

    driver.quit()

    return phone_numbers, real_estate_offices, apartment_info

def extract_price(apartment_str):
    """ '매매' 키워드 오른쪽 문자열을 숫자형으로 변환 """
    match = re.search(r'매매\s*([\d억천만, ]+)', apartment_str)
    if match:
        price_str = match.group(1)
        return convert_price_to_int(price_str)
    return None

def convert_price_to_int(price_str):
    price_str = price_str.replace(',', '').replace(' ', '').strip()
    total_price = 0

    # '억' 단위 처리
    if '억' in price_str:
        parts = re.split(r'억', price_str)
        if parts[0]:
            total_price += int(parts[0]) * 10**8
        price_str = parts[1] if len(parts) > 1 else ''

    # '천' 단위 처리 (천만원)
    if '천' in price_str:
        parts = re.split(r'천', price_str)
        if parts[0]:
            total_price += int(parts[0]) * 10**7
        price_str = parts[1] if len(parts) > 1 else ''

    # '만' 단위 처리
    if '만' in price_str:
        parts = re.split(r'만', price_str)
        if parts[0]:
            total_price += int(parts[0]) * 10**4
        price_str = parts[1] if len(parts) > 1 else ''

    # 남은 숫자 처리 (만원 단위로 가정)
    if price_str and price_str.isdigit():
        total_price += int(price_str) * 10**4

    return total_price if total_price != 0 else None

article_numbers = list(set([
    2504822211, 2504210136, 2505908173, 2505703456, 2503752598, 2501572048, 2502735210, 2502294399,
    2501662043, 2502293027, 2502356205, 2502814467, 2505703905, 2501607928, 2501658024, 2505050574,
    2501996356, 2504771773, 2502179793, 2504023203, 2503791928, 2502618284, 2505796375, 2503086595,
    2504444743, 2503516163, 2504694825, 2502357964, 2502025913, 2502353435, 2503170922
]))

data_list = []
serial_number = 1

for article_no in article_numbers:
    phone_numbers, real_estate_offices, apartment_info = get_data(article_no)
    apartment_str = ' '.join(apartment_info)  # 아파트 정보 통합
    price = extract_price(apartment_str)

    # 중복된 매물번호 체크
    existing_articles = [row[1] for row in data_list]
    if article_no not in existing_articles:
        data_list.append([
            serial_number,
            article_no,
            apartment_str,
            price,
            real_estate_offices,
            phone_numbers
        ])
        serial_number += 1

# 엑셀 저장 경로 설정
path = '/content/drive/MyDrive/python/crawling/hillstate.xlsx'

df = pd.DataFrame(data_list, columns=['일련번호', '매물번호', '아파트 동 호수', '가격', '부동산', '전화번호'])

# 엑셀 파일 저장
df.to_excel(path, index=False)

# 결과 출력
print(f"데이터가 '{path}' 경로에 엑셀 파일로 저장되었습니다!")
print("\n===== 수집된 데이터 =====\n")
print(df.head(10))

데이터가 '/content/drive/MyDrive/python/crawling/hillstate.xlsx' 경로에 엑셀 파일로 저장되었습니다!

===== 수집된 데이터 =====

   일련번호        매물번호                                        아파트 동 호수  \
0     1  2502814467  우장산힐스테이트 115동 13층매매13억 7,000 우장산힐스테이트 115동 13층   
1     2  2503086595          우장산힐스테이트 140동 고층매매15억 우장산힐스테이트 140동 고층   
2     3  2503516163        우장산힐스테이트 115동 16층매매15억 우장산힐스테이트 115동 16층   
3     4  2503752598    우장산힐스테이트 117동 중층매매13억 4,000 우장산힐스테이트 117동 중층   
4     5  2505796375          우장산힐스테이트 140동 고층매매15억 우장산힐스테이트 140동 고층   
5     6  2502353435        우장산힐스테이트 102동 12층매매15억 우장산힐스테이트 102동 12층   
6     7  2505703456    우장산힐스테이트 117동 중층매매13억 4,000 우장산힐스테이트 117동 중층   
7     8  2502293027    우장산힐스테이트 134동 저층매매13억 5,000 우장산힐스테이트 134동 저층   
8     9  2504023203  우장산힐스테이트 113동 11층매매14억 8,000 우장산힐스테이트 113동 11층   
9    10  2501658024    우장산힐스테이트 108동 중층매매14억 2,000 우장산힐스테이트 108동 중층   

           가격                   부동산  \
0  1370000000       [샘터부동산공인중개사사무소]   
1  1500000000       [부동산뉴스공인중개사사무소]  

In [ ]:
#처음에 chatGPT legacy모델로 만든 코드

from selenium import webdriver
from bs4 import BeautifulSoup
import time

def get_data(article_no):
    # 크롬 드라이버 설정 (드라이버 경로 수정 필요)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 브라우저 창 없이 실행
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)

    # 유동적인 URL 설정
    base_url = 'https://new.land.naver.com/complexes/11337?ms=37.556255,126.840704,17&a=APT:ABYG:JGC:PRE&e=RETAIL&ad=true&articleNo='

    url = f"{base_url}{article_no}"

    driver.get(url)

    # 페이지 로딩 대기
    time.sleep(5)

    # 페이지 소스 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 전화번호 크롤링
    phone_numbers = [dd.get_text() for dd in soup.find_all('dd', class_='text text--number')]

    # 부동산 사무소 이름 크롤링
    real_estate_offices = [strong.get_text() for strong in soup.find_all('strong', class_='info_title')]

    # 아파트 동 및 호수 정보 추출
    apartment_info = []
    for h4 in soup.find_all('h4', class_='info_title'):
        title_name = h4.find('strong', class_='info_title_name').get_text() if h4.find('strong', class_='info_title_name') else ''
        floor_info = h4.get_text(strip=True).replace(title_name, '')  # 동 정보 제거 후 층 정보만 남기기
        apartment_info.append(f"{title_name} {floor_info}")

    driver.quit()

    return phone_numbers, real_estate_offices, apartment_info


# 주어진 article 번호 리스트
article_numbers = [
    2504822211, 2504210136, 2505908173, 2505703456, 2503752598, 2501572048, 2502735210, 2502294399,
    2501662043, 2502293027, 2502356205, 2502814467, 2505703905, 2501607928, 2501658024, 2505050574,
    2501996356, 2504771773, 2502179793, 2504023203, 2503791928, 2502618284, 2505796375, 2503086595,
    2504444743, 2503516163, 2504694825, 2502357964, 2502025913, 2502353435, 2503170922
]

# 결과 출력
for article_no in article_numbers:
    phone_numbers, real_estate_offices, apartment_info = get_data(article_no)
    print(f"매물번호: {article_no}")
    print("아파트 동 호수:", apartment_info if apartment_info else "정보 없음")
    print("전화번호:", phone_numbers)
    print("부동산:", real_estate_offices)
    print('-' * 50)

매물번호: 2504822211
아파트 동 호수: ['우장산힐스테이트 117동 저층매매13억', '우장산힐스테이트 117동 저층']
전화번호: ['서울 강서구 내발산동657 우장산힐스테이트 상가1동 102-5호', '02-2658-3080,010-4178-8428']
부동산: ['박사공인중개사사무소']
--------------------------------------------------
매물번호: 2504210136
아파트 동 호수: ['우장산힐스테이트 132동 저층매매13억', '우장산힐스테이트 132동 저층']
전화번호: ['서울 강서구 강서로 348,우장산힐스테이트 상가1동 104-12호', '02-3662-5555,010-6589-2477']
부동산: ['금돼지부동산공인중개사사무소']
--------------------------------------------------
매물번호: 2505908173
아파트 동 호수: ['우장산힐스테이트 137동 저층매매13억 2,000', '우장산힐스테이트 137동 저층']
전화번호: ['서울 강서구 강서로 348,우장산힐스테이트 상가1동 104-12호', '02-3662-5555,010-6589-2477']
부동산: ['금돼지부동산공인중개사사무소']
--------------------------------------------------
매물번호: 2505703456
아파트 동 호수: ['우장산힐스테이트 117동 중층매매13억 4,000', '우장산힐스테이트 117동 중층']
전화번호: ['서울 강서구 내발산동657 우장산힐스테이트 상가1동 102-3', '02-2658-9898,010-6341-5045']
부동산: ['샘터부동산공인중개사사무소']
--------------------------------------------------
매물번호: 2503752598
아파트 동 호수: ['우장산힐스테이트 117동 중층매매13억 4,000', '우장산힐스테이트 117동 중층']
전화번호: ['서울특별시 강서구